In [67]:
## Runs my implementation of Word2Vec
## The input is a text file plus some parameters 
## The pre-processing is performed by functions in TexttoDict

import numpy as np
import theano
import theano.tensor as T
from theano import shared
import nltk
import string
from collections import defaultdict
from TexttoDict import *

rng = np.random

In [ ]:
#---------------------------------------
def initweights(V,N):
    W1 = theano.shared(rng.randn(V,N), name="W1")   # a V x N matrix
    W2 = theano.shared(rng.randn(V,N), name="W2")   # a V x N matrix
    return [W1,W2]

#---------------------------------------
# C is the size of the bag of words
def gettrain(W1,W2):
    
# N is the size of hidden layer
# V is length of dictcut
    V=len(W1.get_value())
    N=len(W1.get_value()[0])

# (x,y) are shared variables for (input, output)
# x is the input bag of words
# y is the target word, the middle word in the bag
    x = T.ivector("x") # a vector
    y = T.ivector("y") # a vector

# This evaluates the hidden and output layers
# This is a linear operation, no logistic function unlike neural nets
# To be more precise we are evaluating these at the level of the hidden layer
    vhidden = W1[x] # a C x N matrix
    vout = W2[y] # a 1 x N matrix

    vhiddenvec= T.mean(vhidden,axis=0)  # an N-vector
    voutvec=vout.reshape((N,)) # an N-vector
    
# Entropy loss function (this is a scalar)
    Entropy = -T.dot(vhiddenvec,voutvec) + T.log(T.exp(T.dot(vhidden,W2.T)).sum())

# Compute the gradient of the cost wrt the weights
    gradW1, gradW2 = T.grad(Entropy, [W1, W2])

# The train function updates the weights and computes the entropy
    train = theano.function(
          inputs=[x,y],
          outputs=[W1,W2,Entropy],
          updates=((W1, W1 - 0.1 * gradW1),(W2, W2 - 0.1 * gradW2)))
   
    return train

#---------------------------------------
def W2Vtrain(inarr,outarr):
    for i in range(len(inarr)):
        W1,W2,ent=train(inarr[i], outarr[i])
    
    return [W1, ent]

This provides the bags of words from the textfile
using functions from TexttoDict.py

In [115]:
W2Vdata=w2vdatafromtext("delorme.txt", contsize=7, trainlines=50000,freqcutoff=1)

We use the functions in this file to prepare for W2V

In [116]:
V=len(W2Vdata[0])
N=10
weights=initweights(V,N)
train=gettrain(weights[0],weights[1])

We now run W2V on the training data

In [117]:
W2Vtrain(W2Vdata[1],W2Vdata[2])

[array([[-0.31743458,  1.13247936, -0.32688617, ...,  1.01424284,
         -1.51509633,  0.0956106 ],
        [ 0.74059688, -0.09327022, -0.71152648, ..., -1.29530879,
         -0.56048464, -1.20894557],
        [ 1.80307889,  0.73352653, -0.09453358, ..., -0.19182782,
         -0.19537542,  0.34890444],
        ..., 
        [ 0.54006604,  0.1307789 ,  0.58469422, ...,  0.43557733,
         -0.0661391 ,  0.60073479],
        [-0.69574132,  0.43231904,  0.54627533, ..., -0.44183667,
         -0.22953882, -1.58285563],
        [ 0.5538179 , -1.61121227, -0.24933756, ...,  0.20676074,
          0.96138345,  0.15373191]]), array(8.694616360398802)]